In [ ]:
import os
import pandas as pd
import librosa.display
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
from pickle import load
model = load(open('SER_MLP_01_scaled.pkl', 'rb'))
scaler = load(open('scaler.pkl', 'rb'))
extract_features=load(open('extract_features.pkl','rb'))

In [ ]:
import librosa

filename = '/content/WhatsApp Audio 2023-06-28 at 14.55.15.wav'  # Your File
duration = 3  # Desired duration of each segment in seconds

# Load the audio file
y, sr = librosa.load(filename)

# Calculate the total duration of the audio file in seconds
total_duration = librosa.get_duration(y=y,sr=sr)

# Calculate the number of segments needed
num_segments = int(total_duration / duration)

# Split the audio file into segments
segments = [[],[]]
count=[0,0,0,0,0,0]
for i in range(num_segments):
    start_time = i * duration
    end_time = (i + 1) * duration
    segment = y[int(start_time * sr):int(end_time * sr)]
    x = extract_features(segment)
    x = scaler.transform((np.array(x)).reshape(1,182))
    y_pred = model.predict(np.array(x))
    #print(y_pred)
    max_index = np.argmax(y_pred)
    acc=y_pred[0][max_index]*100
    #y_pred = np.round_(y_pred)
    if(max_index == 0):
      y_pred="ANGRY"
      count[0]=count[0]+1
    elif(max_index == 1):
      y_pred="DISGUST"
      count[1]=count[1]+1
    elif(max_index == 2):
      y_pred="FEAR"
      count[2]=count[2]+1
    elif(max_index == 3):
      y_pred="HAPPY"
      count[3]=count[3]+1
    elif(max_index == 4):
      y_pred="NEUTRAL"
      count[4]=count[4]+1
    elif(max_index == 5):
      y_pred="SAD"
      count[5]=count[5]+1
    segments[0].append(y_pred)
    segments[1].append(acc)

1/1 [==============================] - 0s 65ms/step


In [ ]:
for i in range(num_segments):
    print(i*3,": 00 -",(i+1)*3,": 00  ->","The Predicted emotion is : ",segments[0][i])

0 : 00 - 3 : 00  -> The Predicted emotion is :  ANGRY -  59.4559383392334 %


In [ ]:
max_emotion=np.argmax(count)
if(max_emotion == 0):
  em="ANGRY"
elif(max_emotion == 1):
  em="DISGUST"
elif(max_emotion == 2):
  em="FEAR"
elif(max_emotion == 3):
  em="HAPPY"
elif(max_emotion == 4):
  em="NEUTRAL"
elif(max_emotion == 5):
  em="SAD"
print("Overall emotion predicted is",em)

Overall emotion predicted is ANGRY - 59.4559383392334 %


In [ ]:
#RECORDED USING MICROPHONE:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 2
RATE = 44100 #sample rate
RECORD_SECONDS = 4
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")
frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()